In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

In [3]:
file_path = 'heart.csv'
data = pd.read_csv(file_path)

In [4]:
app = dash.Dash(__name__)

In [5]:
app.layout = html.Div([
    html.H1("Heart Disease Data Dashboard", style={'textAlign': 'center'}),
    
    html.Div([
        html.Label("Filter by Age Range:"),
        dcc.RangeSlider(
            id='age-slider',
            min=data['age'].min(),
            max=data['age'].max(),
            step=1,
            marks={i: str(i) for i in range(data['age'].min(), data['age'].max()+1, 5)},
            value=[data['age'].min(), data['age'].max()]
        ),
    ], style={'margin': '20px'}),
    
    html.Div([
        html.Label("Filter by Sex:"),
        dcc.Dropdown(
            id='sex-dropdown',
            options=[
                {'label': 'Male', 'value': 1},
                {'label': 'Female', 'value': 0}
            ],
            value=None,
            placeholder="Select a gender"
        )
    ], style={'margin': '20px'}),
    
    html.Div([
        html.Label("Filter by Target (Heart Disease):"),
        dcc.Dropdown(
            id='target-dropdown',
            options=[
                {'label': 'Disease', 'value': 1},
                {'label': 'No Disease', 'value': 0}
            ],
            value=None,
            placeholder="Select a target outcome"
        )
    ], style={'margin': '20px'}),
    
    dcc.Graph(id='scatter-plot', style={'height': '600px'}),
    
    dcc.Graph(id='histogram', style={'height': '600px'}),
])


In [7]:
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('histogram', 'figure')],
    [Input('age-slider', 'value'),
     Input('sex-dropdown', 'value'),
     Input('target-dropdown', 'value')]
)
def update_graphs(age_range, sex, target):
    # Filter data based on inputs
    filtered_data = data[
        (data['age'] >= age_range[0]) & (data['age'] <= age_range[1])
    ]
    if sex is not None:
        filtered_data = filtered_data[filtered_data['sex'] == sex]
    if target is not None:
        filtered_data = filtered_data[filtered_data['target'] == target]
    
    # Scatter plot
    scatter_fig = px.scatter(
        filtered_data,
        x='age',
        y='thalach',
        color='target',
        title="Age vs. Max Heart Rate (thalach)",
        labels={'thalach': 'Max Heart Rate', 'age': 'Age'}
    )
    
    # Histogram
    hist_fig = px.histogram(
        filtered_data,
        x='cp',
        color='target',
        barmode='group',
        title="Chest Pain Type Distribution",
        labels={'cp': 'Chest Pain Type'}
    )
    
    return scatter_fig, hist_fig


In [8]:
if __name__ == '__main__':
    app.run_server(debug=True)